# Solutions


## 1. ANN

Since we have created DataLoaders we now can iterate over them.
In the the next cell, there is the loop for iterating over `trainset` and inspect it. Do the same with `testset`!

In [ ]:
import matplotlib.pyplot as plt

for data in testset:
    batch_of_images = data[0]
    print(batch_of_images.size())
    batch_of_labels = data[1]
    print(batch_of_labels.size())

    fig = plt.figure()
    fig.set_size_inches(18.5, 10.5, forward=True)
    for i, (image, label) in enumerate(zip(batch_of_images,batch_of_labels)):
        plt.subplot(2,5,i+1)
        plt.imshow(image.view(28,28), cmap='gray', interpolation='none')
        plt.title(f"Ground Truth: {label}")
    fig

    break

Now verify the Network with one of data from our dataset. 
Hint: Take a batch from out `trainset` using `next(iter(trainset))`.

In [ ]:
X = next(iter(trainset))[0][0]
print('Input:')
plt.imshow(X.view(28,28))
plt.show()


X = X.view(-1,28*28) # neural network wants this to be flattened
output = net(X)
print(f'Output: {output}')

In [ ]:
NUMBER_OF_HIDDEN_LAYERS = 5

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 64)
        self.hidden_layers = nn.Sequential()
        #### YOUR CODE HERE
        for i in range(NUMBER_OF_HIDDEN_LAYERS):
            self.hidden_layers.append(nn.Linear(64, 64))
        self.fc_n = nn.Linear(64, 10) 

    # passing our data through the layers + activations
    def forward(self, x):
        x = F.relu(self.fc1(x)) 
        #### YOUR CODE HERE
        for hl in self.hidden_layers:
            x = F.relu(hl(x))
        x = self.fc_n(x)
        return F.log_softmax(x, dim=1) 

net = Net()
print(net)


X = torch.randn((28,28)) # create random 28x28 image
print('Input:')
plt.imshow(X.view(28,28))
plt.show()


X = X.view(-1,28*28) # neural network wants this to be flattened
output = net(X)
print(f'Output: {output}')

## 2. Autoencoders

Modify the architecture of the Autoencoder by adding `nn.BatchNorm1d(out)` layer after each Linear layer, where `out` parameter should be equal the size of output of previous Linear layer. 

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        # Role of encoder: repeatedly reduce the size
        # N - batch size
        # input dimensions: 28x28 = 784
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128), # (N, 784) -> (N, 128)
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(64, 12),
            nn.BatchNorm1d(12),
            nn.ReLU(),
            nn.Linear(12, 3) # -> N, 3
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.BatchNorm1d(12),
            nn.ReLU(),
            nn.Linear(12, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Linear(128, 28 * 28),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

Similarly to the example with Linear Autoencoder - add some Normalization layers. REMARK - use `nn.BatchNorm2d(out)` after each Conv2d or Conv2dTranspose layer.

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()        
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=2, padding=1), # -> N, 16, 14, 14
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride=2, padding=1), # -> N, 32, 7, 7
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 64, 7, stride=1, padding=0), # -> N, 64, 1, 1
        )
        
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, 7, stride=1, padding=0), #  -> N, 32, 7, 7
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 3, stride=2, padding=1, output_padding=1), # -> N, 16, 14, 14
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 1, 3, stride=2, padding=1, output_padding=1), # -> N, 1, 28, 28 
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


Try to design the Autoencoder that will be able to process our dataset with Photons. The rest (training loop, loss, optimizers) are already coded in next cells. Your role is to put layers into the encoder and decoder!


In [ ]:
class Autoencoder_Linear(nn.Module):
    def __init__(self):
        super().__init__()        
        self.encoder = nn.Sequential(
            nn.Linear(6, 6), 
            nn.ReLU(),
            nn.Linear(6, 5),
            nn.ReLU(),
            nn.Linear(5, 3),
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(3, 5),
            nn.ReLU(),
            nn.Linear(5, 6),
            nn.ReLU(),
            nn.Linear(6, 6)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded, encoded

## 3. GANs
